# New Mexico Redistricting Problem

## Import Libraries

In [1]:
import sys
import numpy as np
import pandas as pd

solver = 'appsi_highs'

import pyomo.environ as pyo
SOLVER = pyo.SolverFactory(solver)

assert SOLVER.available(), f"Solver {solver} is not available."

## Data Preprocessing

In [2]:
# Read the CSV file into a pandas DataFrame
filename = 'Gerrymandering.csv'
data = pd.read_csv(filename)

## Construct the Model

In [3]:
def redistricting_model(data):
    '''
    args: 
        data: DataFrame
    return: 
        model: pyo.ConcreteModel
    '''
    # Create a model
    model = pyo.ConcreteModel("New Mexico Redistricting")

    # Sets of districts(i) and counties(j)
    model.i = pyo.RangeSet(3)
    model.j = pyo.RangeSet(len(data))

    # Decision variables
    model.x = pyo.Var(model.i, model.j, domain=pyo.Binary)

    # Objective function
    @model.Objective(sense=pyo.maximize)
    def maximize_votes(model):
        return sum(model.x[2, j] * data["D_j - R_j"][j - 1] for j in model.j)

    # Constraints
    # Constraint for counties
    @model.Constraint(model.j)
    def constraint_counties(model, j):
        return sum(model.x[i, j] for i in model.i) == 1

    # Constraint for districts
    @model.Constraint(model.i)
    def constraint_districts(model, i):
        return sum(model.x[i, j] for j in model.j) >= 1

    # Constraint for votes
    @model.Constraint(model.i)
    def constraint_votes(model, i):
        return sum(model.x[i, j] * data["D_j - R_j"][j - 1] for j in model.j) >= 100
    
    # return the finished model
    return model


## Solve the Model

In [4]:
def solve_model(model):
    '''
    args: 
        pyo.ConcreteModel
    '''
    SOLVER.solve(model)
    model.display()

In [5]:
model = redistricting_model(data)
solve_model(model)

Model New Mexico Redistricting

  Variables:
    x : Size=99, Index=x_index
        Key     : Lower : Value              : Upper : Fixed : Stale : Domain
         (1, 1) :     0 :               -0.0 :     1 : False : False : Binary
         (1, 2) :     0 :                0.0 :     1 : False : False : Binary
         (1, 3) :     0 :               -0.0 :     1 : False : False : Binary
         (1, 4) :     0 :                0.0 :     1 : False : False : Binary
         (1, 5) :     0 :               -0.0 :     1 : False : False : Binary
         (1, 6) :     0 :               -0.0 :     1 : False : False : Binary
         (1, 7) :     0 :                1.0 :     1 : False : False : Binary
         (1, 8) :     0 :                0.0 :     1 : False : False : Binary
         (1, 9) :     0 :               -0.0 :     1 : False : False : Binary
        (1, 10) :     0 :                0.0 :     1 : False : False : Binary
        (1, 11) :     0 :                1.0 :     1 : False : Fal

## Result Visualization

In [6]:
def visualization(model):
    '''
    args: 
        pyo.ConcreteModel
    '''
    # initialize the map
    district_county_map1 = {i: [] for i in model.i}
    district_county_map2 = {i: [] for i in model.i}
    # build the map
    for i in model.i:
        for j in model.j:
            if pyo.value(model.x[i, j]) == 1:
                district_county_map1[i].append(j)
                district_county_map2[i].append(data['County'][j-1])
    # show the map of version 1
    print("Show in index of counties: ")
    for i in district_county_map1:
        print(f"Counties in district {i}: {district_county_map1[i]}")
    print()
    # show the map of version 2
    print("Show in names of counties: ")
    for i in district_county_map2:
        print(f"Counties in district {i}: {district_county_map2[i]}")

In [7]:
visualization(model)

Show in index of counties: 
Counties in district 1: [7, 11, 14, 22, 32, 33]
Counties in district 2: [4, 8, 10, 15, 16, 17, 18, 19, 24, 26, 27, 29, 30]
Counties in district 3: [1, 2, 3, 5, 6, 9, 12, 13, 20, 21, 23, 25, 31]

Show in names of counties: 
Counties in district 1: ['DeBaca', 'Guadalupe', 'Lea', 'Rio Arriba', 'Union', 'Valencia']
Counties in district 2: ['Cibola', 'Dona Ana', 'Grant', 'Lincoln', 'Los Alamos', 'Luna', 'McKinley', 'Mora', 'Sandoval', 'San Miguel', 'Santa Fe', 'Socorro', 'Taos']
Counties in district 3: ['Bernalillo', 'Catron', 'Chaves', 'Colfax', 'Curry', 'Eddy', 'Harding', 'Hidalgo', 'Otero', 'Quay', 'Roosevelt', 'San Juan', 'Torrance']
